# Morphology quality checks
Copyright (c) 2025 Open Brain Institute

Authors: Michael W. Reimann

last modified: 08.2025

## Summary
This notebook runs a barrage of standard quality tests on selected reconstructed morphology files.
It then displays the results, i.e., how many and which morphologies fail tests.
For details, see the [README](README.md).

## Imports and setting up platform authentication

Please follow the displayed instructions to authenticate.

In [ ]:
from obi_auth import get_token
from entitysdk.client import Client
try:
    from entitysdk.models import ReconstructionMorphology
    entity_tp_str = "reconstruction-morphology"
except ImportError:
    from entitysdk.models import CellMorphology as ReconstructionMorphology
    entity_tp_str = "cell-morphology"
import pandas as pd

import neurom
import os
import tqdm

from ipywidgets import widgets
from neurom.check.runner import CheckRunner
from matplotlib import pyplot as plt
from obi_notebook import get_projects
from obi_notebook import get_entities


## Selection of morphologies and download

Morphologies will be downloaded to `analysis_morphologies`, then loaded and analyzed

#### Project selection
As a first step we select one of the projects we have access to that the morphologies are associated with. If the morphologies of interest are part of the public OBI assets, any project can be selected.

In [ ]:
token = get_token(environment="production", auth_mode="daf")
project_context = get_projects.get_projects(token)

#### Morphology selection

Next, we select the morphologies. If you already know the unique identifiers of the morphologies of interest, paste them below as a list (!) into line 4 of the next cell.

Otherwise, a widget for morphology selection will be created that allows you to filter and select multipe morphologies.

In [ ]:
client = Client(environment="production", project_context=project_context, token_manager=token)

# Optional: Download using unique ID
entity_IDs = "<MORPHOLOGY-IDs>"  # <<< FILL IN A LIST OF UNIQUE MORPHOLOGY IDENTIFIERS HERE


if entity_IDs != "<MORPHOLOGY-IDs>":
    morph_ids = [entity_IDs]
else:
# Alternative: Select from a table of entities
    morph_ids = []
    morph_ids = get_entities.get_entities(entity_tp_str, token, morph_ids,
                                            project_context=project_context,
                                            multi_select=True,
                                            page_size=100)

#### Selection of morphology file type

Moprphologies can be stored in different file formats. While in principle conversion between the formats can be done with no (or minimal) loss, in some cases differences can still occur. Therefore, we have to explicitly select which of the available formats to use in the widget below.

The numbers after the file format indicate how many of the selected neuron morphologies are available in each format. If a format is not available for a selected morphology, it will be skipped.

In [ ]:
fetched = [client.get_entity(entity_id=morph_id_, entity_type=ReconstructionMorphology) for morph_id_ in morph_ids]

asset_types = {}
for fetched_ in fetched:
    for asset in fetched_.assets:
        tp_name = str(asset.content_type)
        asset_types[tp_name] = asset_types.setdefault(tp_name, 0) + 1

asset_type_str = [(f"{k}: {v} / {len(fetched)}", k) for k, v, in asset_types.items()]

asset_type_sel = widgets.Dropdown(options=asset_type_str)
display(asset_type_sel)

#### Fetch morphologies
The selected morphology files are copied to the local system and loaded.

In [ ]:
all_assets = [[asset for asset in fetched_.assets if str(asset.content_type)==asset_type_sel.value]
              for fetched_ in fetched]

morphologies_root = "analysis_morphologies"
if not os.path.exists(morphologies_root):
    os.makedirs(morphologies_root)

nm_morphs = {}
for asset_, morph_id_, fetched_ in tqdm.tqdm(list(zip(all_assets, morph_ids, fetched))):
    if len(asset_) >= 1:
        out_fn = client.download_file(
            entity_id=morph_id_,
            entity_type=ReconstructionMorphology,
            asset_id=asset_[0],
            output_path=morphologies_root,
            project_context=project_context
        )
        nm_morphs.setdefault("_morphology", []).append(neurom.load_morphology(out_fn))
        nm_morphs.setdefault("Region", []).append(fetched_.brain_region.acronym)
        nm_morphs.setdefault("Morphology type", []).append(fetched_.mtypes[0].pref_label)
        nm_morphs.setdefault("Created by", []).append(fetched_.created_by.pref_label)
        nm_morphs.setdefault("Name", []).append(fetched_.name)
nm_morphs = pd.DataFrame(nm_morphs)


#### Execute tests

In [ ]:
# morphology_checks: concern the valid structure of a morphology
# options: set the tolerance parameters for the checks

config = {
    "checks": {
        "morphology_checks": [
            "has_axon",
            "has_basal_dendrite",
            "has_apical_dendrite",
            "has_no_jumps",
            "has_no_fat_ends",
            "has_nonzero_soma_radius",
            "has_all_nonzero_neurite_radii",
            "has_all_nonzero_section_lengths",
            "has_all_nonzero_segment_lengths",
            "has_no_flat_neurites",
            "has_nonzero_soma_radius",
            "has_no_narrow_start",
            "has_no_dangling_branch",
        ]
    },
    "options": {
        "has_nonzero_soma_radius": 0.0,
        "has_all_nonzero_neurite_radii": 0.007,
        "has_all_nonzero_segment_lengths": 0.01,
        "has_all_nonzero_section_lengths": 0.01,
    },
}
# create a CheckRunner object by providing the configuration dict
check_runner = CheckRunner(config)

def run_wrapper(_morph):
    try:
        res = check_runner._check_loop(_morph, "morphology_checks")
        res = pd.Series(res[1])
        res["Test ran to completion"] = True
    except:
        res = pd.Series({"Test ran to completion": False})
    return res

res = nm_morphs["_morphology"].apply(run_wrapper).fillna(False)

#### Display results statistics

In [ ]:
fig = plt.figure(figsize=(5, 3))
ax = fig.add_axes([0.3, 0.1, 0.65, 0.9])
ttl = len(res) - res.isna().sum(axis=0)
passed = res.astype(int).sum(axis=0)
ax.barh(range(len(ttl)), ttl, height=0.9, color=[0.8, 0.8, 0.8], label="Run")
ax.barh(range(len(passed)), passed, height=0.5, color=[0.1, 0.6, 0.1], label="Passed")
ax.barh(range(len(passed)), ttl - passed, height=0.5, left=passed, color=[0.8, 0.3, 0.3], label="Failed")
ax.set_yticks(range(len(passed)))
ax.set_yticklabels(passed.index)
ax.set_xlabel("# Morphologies")
ax.set_frame_on(False)
plt.legend()
plt.show()

pick_test_wdgt = widgets.Dropdown(options=res.columns)
pick_col_wdgt = widgets.Dropdown(options=nm_morphs.drop(columns="_morphology").columns,
                                 value="Name")
output = widgets.HTML()
display(widgets.HTML(
    value="<b>Pick a test to list morphologies failing it</b>"
))
display(pick_test_wdgt)
display(pick_col_wdgt)
display(output)

def display_func(sel_test_name, sel_col):
    col = res[sel_test_name]
    morphs = nm_morphs[sel_col][col == False].values
    output.value=", ".join(list(morphs))
    print(output.value)
_ = widgets.interactive(display_func, sel_test_name=pick_test_wdgt, sel_col=pick_col_wdgt)

